# Building a ChatGPT-like Interface for Azure PostgreSQL Data

This notebook demonstrates how to build a natural language interface to query Azure PostgreSQL data using Microsoft's **Semantic Kernel** and **Azure OpenAI**.  
We will walk through:
- Setting up environment variables
- Connecting to Azure PostgreSQL
- Initializing Semantic Kernel
- Defining "skills" for SQL generation and answer formatting
- Running queries end-to-end

In [3]:
!pip install psycopg2

  Using cached psycopg2-2.9.10-cp310-cp310-win_amd64.whl.metadata (5.0 kB)
Using cached psycopg2-2.9.10-cp310-cp310-win_amd64.whl (1.2 MB)


**--- Load Environment Variables ---**

- We use python-dotenv to load sensitive values (DB credentials, API keys) from a `.env` file.  
- This ensures credentials are not hardcoded in the notebook.


In [48]:
import os
from dotenv import load_dotenv
import psycopg2
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
import json
import asyncio

# Load environment variables
load_dotenv(override=True)

True

**Database Connection**

In [49]:
# --- Connect to Azure PostgreSQL ---
# psycopg2 is used for PostgreSQL connection and transaction management.

try:
    conn = psycopg2.connect(
        host=os.getenv("PGHOST"),
        dbname=os.getenv("PGDATABASE"),
        user=os.getenv("PGUSER"),
        password=os.getenv("PGPASSWORD"),
        port=os.getenv("PGPORT"),
    )
    cursor = conn.cursor()
except psycopg2.Error as e:
    print(f"Failed to connect to database: {e}")
    raise

In [50]:
# Semantic Kernel setup
# --- Initialize Semantic Kernel ---
# Semantic Kernel orchestrates AI workflows by combining:
#  - Azure OpenAI (for natural language understanding)
#  - Custom "skills" (for SQL generation & answer formatting)

kernel = sk.Kernel()
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_KEY")
try:
    kernel.add_service(
        AzureChatCompletion(
            service_id="azure-openai",
            deployment_name=deployment_name,
            endpoint=endpoint,
            api_key=api_key,
        )
    )
except Exception as e:
    print(f"Failed to initialize AzureChatCompletion: {e}")
    raise

**---Test 1 :  Define Semantic Kernel Skills to convert NL to SQL command ---**

- Skills are modular functions defined via prompts or code.  
- Here, we define:  
  1. **GenerateSql** → Converts natural language into SQL queries.  
  2. **AnswerQuestion** → Provides the SQL query or answers 'Not enough information'.  

In [51]:
from semantic_kernel import Kernel

# --- Create SQL generator function ---
sql_skill1 = kernel.add_function(
    plugin_name="SqlSkill",
    function_name="GenerateSql",
    prompt="""
    You are a SQL generator.
    Convert the user natural language question into a SQL query for the 'iris' table.
    Return only the SQL query without explanation.
    """
)

# --- Create Answering function ---
answer_skill1 = kernel.add_function(
    plugin_name="AnswerSkill",
    function_name="AnswerQuestion",
    prompt="""
    You are a helpful assistant.
    Answer the user question based only on the provided database results.
    If the answer cannot be determined, say "Not enough information."
    """
)

In [52]:
async def main():
    user_question = "Show all iris records where species is setosa"

    # Generate SQL
    sql_query = await sql_skill1.invoke(user_question)
    print("Generated SQL:", sql_query)

    # Suppose SQL results are fetched
    sql_results = [
        {"sepal_length": 5.1, "sepal_width": 3.5, "petal_length": 1.4, "petal_width": 0.2, "species": "setosa"}
    ]

    # Get answer
    answer = await answer_skill1.invoke(
        f"What is the average petal length?",
        context={"results": sql_results}
    )
    print("Answer:", answer)



In [53]:
from semantic_kernel.functions.kernel_arguments import KernelArguments

In [54]:
user_question = "Show all iris records where species is setosa"

# Generate SQL
sql_query = await sql_skill1.invoke(
    kernel,
    KernelArguments(input=user_question)  # 👈 wrap your input
)

print("Generated SQL:", sql_query)

Generated SQL: SELECT * FROM iris WHERE species = 'setosa';


**---Test 2 : Converts NL Query->SQL-> Human readable answers ---**

**--- Define Semantic Kernel Skills ---**
- Skills are modular functions defined via prompts or code.  
- Here, we define:  
  1. **GenerateSql** → Converts natural language into SQL queries.  
  2. **AnswerQuestion** → Formats raw query results into human-readable answers.  


In [55]:
# Define SQL and Answer functions
sql_skill = kernel.add_function(
    plugin_name="SqlSkill",
    function_name="GenerateSql",
    prompt="""
    You are a SQL generator for a PostgreSQL database.
    Convert the user natural language question into a SQL query for the 'iris' table with columns: sepal_length, sepal_width, petal_length, petal_width, species.
    Return only the SQL query without explanation.
    For average calculations, use ROUND(AVG(column)::numeric, 1) to handle double precision.
    Examples:
    - For "average sepal length by species": "SELECT species, ROUND(AVG(sepal_length)::numeric, 1) AS avg_sepal_length FROM iris GROUP BY species;"
    - For "list virginica flowers with petal width greater than 2.0": "SELECT sepal_length, sepal_width, petal_length, petal_width FROM iris WHERE species = 'virginica' AND petal_width > 2.0;"
    User Question: {{$input}}
    """
)
answer_skill = kernel.add_function(
    plugin_name="AnswerSkill",
    function_name="AnswerQuestion",
    prompt="""
    You are a helpful assistant.
    Answer the user question based only on the provided database results in JSON format.
    If the results are empty, say "No data available to answer the question."
    For averages, use the format: "The average sepal length is X for setosa, Y for versicolor, and Z for virginica."
    For lists, use: "There are N <species> flowers with <condition>: (x1, x2, x3, x4), ..."
    Numerical values should be rounded to one decimal place.
    User Question: {{$input}}
    SQL Results: {{$results}}
    """
)

In [ ]:
# --- Orchestration Function ---
# The ask_db function:
#   1. Takes a natural language question
#   2. Uses the SQL skill to generate a SQL query
#   3. Executes the SQL query against Azure PostgreSQL
#   4. Uses the Answer skill to turn results into a friendly response
#   5. Returns the final answer to the user

async def ask_db(question):
    print(f"Processing question: {question}")
    
    # Generate SQL query
    sql_skill = kernel.get_function("SqlSkill", "GenerateSql")
    try:
        sql_query_result = await sql_skill.invoke(kernel, input=question)
        sql_query = str(sql_query_result)
        print(f"Generated SQL: {sql_query}")
    except Exception as e:
        return f"Error generating SQL: {e}"

    # Execute SQL query
    try:
        cursor.execute(sql_query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        sql_results = [dict(zip(columns, row)) for row in results]
        print(f"SQL Results: {sql_results}")
        conn.commit()  
    except psycopg2.Error as e:
        conn.rollback() 
        return f"Database error: {e}"

    # Answer the question
    answer_skill = kernel.get_function("AnswerSkill", "AnswerQuestion")
    try:
        answer = await answer_skill.invoke(
            kernel,
            input=question,
            results=json.dumps(sql_results)
        )
        return str(answer)
    except Exception as e:
        return f"Error generating answer: {e}"

In [ ]:

async def run_queries():
    print(await ask_db(" what speicies does this belong to 6.8	3.2	5.9	2.3? "))
    print(await ask_db("List all virginica flowers with petal width greater than 2.0"))

In [ ]:

await run_queries()

Processing question:  what speicies does this belong to 6.8	3.2	5.9	2.3? 
Generated SQL: SELECT species FROM iris WHERE sepal_length = 6.8 AND sepal_width = 3.2 AND petal_length = 5.9 AND petal_width = 2.3;
SQL Results: [{'species': 'virginica'}]
The species of the flower with the characteristics mentioned is virginica.
Processing question: List all virginica flowers with petal width greater than 2.0
Generated SQL: SELECT sepal_length, sepal_width, petal_length, petal_width FROM iris WHERE species = 'virginica' AND petal_width > 2.0;
SQL Results: [{'sepal_length': 6.3, 'sepal_width': 3.3, 'petal_length': 6.0, 'petal_width': 2.5}, {'sepal_length': 7.1, 'sepal_width': 3.0, 'petal_length': 5.9, 'petal_width': 2.1}, {'sepal_length': 6.5, 'sepal_width': 3.0, 'petal_length': 5.8, 'petal_width': 2.2}, {'sepal_length': 7.6, 'sepal_width': 3.0, 'petal_length': 6.6, 'petal_width': 2.1}, {'sepal_length': 7.2, 'sepal_width': 3.6, 'petal_length': 6.1, 'petal_width': 2.5}, {'sepal_length': 6.8, 'sep

## Conclusion
We successfully built a ChatGPT-like interface to query Azure PostgreSQL data.  
By leveraging Semantic Kernel, we combined:
- LLM-based natural language understanding
- SQL query generation
- Database connectivity
- Human-readable responses  

This pipeline can be extended to other datasets and domains, enabling Natural Language Querying (NLQ) for business users.
